# 概要
この前Twitter APIに触ってみた。今回はそれに引き続き、知識が横展開出来そうなGmailAPIを触ってみることにした。<br>
[ここ](https://console.developers.google.com/flows/enableapi?apiid=gmail&pli=1)で認証情報を作成

#### インストール
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

##### 参考URL
* [公式Document](https://developers.google.com/gmail/api/quickstart/python)
* [SCOPEの設定](https://developers.google.com/gmail/api/auth/scopes)


#### QuickStart
公式の[Set up the sample](https://developers.google.com/gmail/api/quickstart/python)を実行するとエラーが出た。<br>
実行環境がC:\Users配下であったため、\U以降の文字列がエスケープシーケンスとして扱われた。<br>
指定パスを書き換えてプレフィックスに'r'を付ければok。<br>
実行するとGoogleアカウントを選択する画面が出るので選択する。<br>
するとブラウザが開く<br>

「このアプリは確認されていません。このアプリは、Google による確認が済んでいません。<br>
よく知っている信頼できるデベロッパーの場合に限り続行してください。」と表示される。<br>
下の方にリンク「アプリ名（安全ではないページ）に移動」と表示されているのでリンククリック。<br>
アプリケーションにアクセス許可を与える確認メッセージが表示される。

「The authentication flow has completed, you may close this window.」と表示されたら開発準備完了。

#### Serviceインスタンス作成

In [57]:
def getServiceInstance():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                r'C:\Users\XXXXXXX\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

#### Draft

In [58]:
import base64
def draft(service, to_address, subject, body):
    
    message = MIMEMultipart()
    message['to'] = to_address
    message['from'] = 'me'
    message['subject'] = subject

    msg = MIMEText(body)
    message.attach(msg)

    raw = base64.urlsafe_b64encode(message.as_bytes())
    raw = raw.decode()
    message_body = {'raw': raw}

    create_draft(service, 'me', message_body)

In [62]:
Myaddress='XXXXXX@gmail.com'
message = create_message(Myaddress, Myaddress, 'subject', 'Hello')
draft(getServiceInstance(),Myaddress,"test",'本文')

Draft id: r-6700808622318172887
Draft message: {'id': '16bf5fbc5bbb0706', 'threadId': '16bf5fbc5bbb0706', 'labelIds': ['DRAFT']}


#### Get

In [63]:
def getMessageList(DateFrom,DateTo,MessageFrom):
        service = getServiceInstance()

        MessageList = []

        query = ''

        if DateFrom != None and DateFrom !="":
            query += 'after:' + DateFrom + ' '
        if DateTo != None  and DateTo !="":
            query += 'before:' + DateTo + ' '
        if MessageFrom != None and MessageFrom !="":
            query += 'From:' + MessageFrom + ' '

        messageIDlist = service.users().messages().list(userId='me',maxResults=10,q=query).execute()
        if messageIDlist['resultSizeEstimate'] == 0: 
            print("Message is not found")
            return MessageList
        for message in messageIDlist['messages']:
            row = {}
            row['ID'] = message['id']
            MessageDetail = service.users().messages().get(userId='me',id=message['id']).execute()
            for header in MessageDetail['payload']['headers']:
                if header['name'] == 'Date':
                    row['Date'] = header['value'] 
                elif header['name'] == 'From':
                    row['From'] = header['value']
                elif header['name'] == 'Subject':
                    row['Subject'] = header['value']
            MessageList.append(row)
        return MessageList

In [64]:
messages = getMessageList(DateFrom='2019-01-01',DateTo='2019-02-01',MessageFrom='nikkei-news@mx.nikkei.com')
for message in messages:
        print(message['Subject'])

アマゾン、純利益63％増　初の30億ドル台
IoT機器、防御を義務化　サイバー攻撃入り口封じ
ゴーン元会長、潔白主張　強気崩さず「正当な決裁」
ゴーン元会長、潔白主張　強気崩さず「正当な決裁」
日産ゴーン元会長、逮捕後初のインタビュー
アップル、中国発のスマホ不振　19年も減収避けられず
アップル、10～12月0.5％減益　９四半期ぶり減収減益
「１円スマホ」駆け込み販売　セット値引き禁止前に
景気回復「戦後最長の可能性」　１月の月例経済報告
企業の格下げ　 世界で増加


### 感想
取り合えずgmailの下書きと受信トレイの検索をやってみた。
Python2系と3系の違いから、コピペで動かないような所があった。
Pythonは便利なのだが、インストールするべきライブラリが多すぎて詳細なデバッグが難しいように感じた。
大体StackOverFlowとか公式githubのIssueの方法をトライアンドエラーで動かして、動作したらOKとしているが、
こんなことは現在の職場でやってたら当然ながらレビューは通らないと思われる。
企業としてプログラム製品の品質を担保するとき、Pythonではどのようなアプローチがされているのか興味ある。

後は気になるのがLINE APIとgoogleドライブやスプレッドシートなどのAPIかな。
ちょっとやりたいKaggleコンペがあったので、今回はこのへんで。